In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [3]:
from transformers import CLIPVisionModel, AutoTokenizer, CLIPTextModel

model = "openai/clip-vit-base-patch16"
tokenizer = AutoTokenizer.from_pretrained(model)
image_encoder = CLIPVisionModel.from_pretrained(model)
text_encoder = CLIPTextModel.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [28]:
import os
path = "/root/DeepAccident/data/DeepAccident_data/type1_subtype1_accident/ego_vehicle/Camera_Front/Town01_type001_subtype0001_scenario00008"
img_name = "{}_{:03d}.jpg".format("Town01_type001_subtype0001_scenario00008",1)
img_path = os.path.join(path,img_name)
from torchvision.io import read_image
from torchvision import transforms
img = read_image(img_path)/255
transform = transforms.Compose([
            transforms.Resize((224, 224)),     
        ])
sample = transform(img)

In [32]:
inputs = tokenizer(["collision"], padding=True, return_tensors="pt")
text_outputs = text_encoder(**inputs)['last_hidden_state'].mean(1)
vision_outputs = image_encoder(pixel_values=sample.unsqueeze(0))['last_hidden_state'].mean(1)

In [69]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
img_name = "{}_{:03d}.jpg".format("Town01_type001_subtype0001_scenario00008",1)
img_path = os.path.join(path,img_name)
image1 = Image.open(img_path)
img_name = "{}_{:03d}.jpg".format("Town01_type001_subtype0001_scenario00008",58)
img_path = os.path.join(path,img_name)
image2 = Image.open(img_path)
img_name = "{}_{:03d}.jpg".format("Town01_type001_subtype0001_scenario00008",60)
img_path = os.path.join(path,img_name)
image3 = Image.open(img_path)
img_name = "{}_{:03d}.jpg".format("Town01_type001_subtype0001_scenario00008",66)
img_path = os.path.join(path,img_name)
image4 = Image.open(img_path)

inputs = processor(text=["no collision and accident","collision","collision imminent"], images=[image1,image2,image3,image4], return_tensors="pt", padding=True)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [4]:
from PIL import Image
import requests

from transformers import AutoImageProcessor, ViTMAEForPreTraining
processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-base')
model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-base')
path = "/root/DeepAccident/data/DeepAccident_data/type1_subtype1_accident/ego_vehicle/Camera_Front/Town01_type001_subtype0001_scenario00008"
img_name = "{}_{:03d}.jpg".format("Town01_type001_subtype0001_scenario00008",1)
img_path = os.path.join(path,img_name)
image = Image.open(img_path)
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

preprocessor_config.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/448M [00:00<?, ?B/s]

In [15]:
outputs = model.vit(inputs['pixel_values'])
latent = outputs.last_hidden_state
ids_restore = outputs.ids_restore
mask = outputs.mask

decoder_outputs = model.decoder(latent, ids_restore)
i = model.unpatchify(decoder_outputs['logits'])
# model.decoder(outputs.last_hidden_state,outputs.ids_restore)

In [22]:
opt = model.vit(inputs['pixel_values'])

dict_keys(['pixel_values'])

In [19]:
import matplotlib.pyplot as plt
# plt.imshow(i.squeeze(0).detach().numpy())
# plt.show()
from torchvision.utils import save_image
save_image(i.data[0],"./test.png")
save_image(inputs['pixel_values'].data[0],"./test1.png")